# Testing the lambda handler

In [46]:
%load_ext autoreload
%autoreload 2
from lambda_function import lambda_handler, open_icechunk_repo

import earthaccess
import os
import xarray as xr
import zarr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
os.environ['SECRET_ARN'] = 'arn:aws:secretsmanager:us-west-2:444055461661:secret:mursst_lambda_edl_credentials-9dKy1C'
os.environ['S3_BUCKET_NAME'] = 'mursst-cmr-notifications'

In [48]:
#lambda_handler({})

# Testing the store

In [ ]:
earthaccess.login()
ea_creds = earthaccess.get_s3_credentials(daac='PODAAC')
bucket = 'nasa-eodc-public'
store_name = "MUR-JPL-L4-GLOB-v4.1-virtual-v1-p2"

In [ ]:
repo = open_icechunk_repo(bucket, store_name, ea_creds)
session = repo.readonly_session(branch="main")
xds = xr.open_zarr(session.store, zarr_version=3, chunks={}, consolidated=False)

In [45]:
xds.sel(lat=slice(47.6, 47.7), lon=slice(-122.4, -122.3)).analysed_sst.mean().values

array(284.17548539)

# Deleting data

In [31]:
session = repo.writable_session(branch="main")
store = session.store
variables = ["analysed_sst", "analysis_error", "sea_ice_fraction", "mask", "time"]
resize = 364
for var in variables:
    # Open your array
    group = zarr.open_group(store)
    array = group[var]
    
    # Truncate the array to remove time indices > 364
    # This effectively removes chunks for indices 365, 366, etc.
    new_shape = list(array.shape)
    new_shape[0] = 364  # Assuming time is the first dimension
    array.resize(new_shape)

/opt/conda/lib/python3.11/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


In [38]:
# DANGER!
# session.commit("Removed data for time > 2025-05-31")

'44B9XPA2C0QH6FVT5AF0'